In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt

# File path to the data file
filename = 'Tesla.csv'

# Load and preprocess data
def load_and_preprocess_data(filename):
    # Read the CSV file
    data = pd.read_csv(filename)
    
    # Extract the year from the 'MM/DD/YYYY' date format
    data['Year'] = pd.to_datetime(data['Date'], format='%m/%d/%Y').dt.year
    
    # Convert the 'Close/Last' column to numeric, handling any currency symbols
    data['Close/Last'] = data['Close/Last'].replace('[\$,]', '', regex=True).astype(float)
    
    # Extract the years and prices as separate lists
    dates = data['Year'].values
    prices = data['Close/Last'].values
    
    return dates, prices

# Define and fit the Support Vector Regression (SVR) models
def train_svr_models(dates, prices):
    # Reshape dates for the model (n_samples, n_features) format
    dates = np.reshape(dates, (len(dates), 1))
    
    # Instantiate three SVR models with different kernels
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)  # Radial Basis Function kernel
    svr_lin = SVR(kernel='linear', C=1e3)          # Linear kernel
    svr_poly = SVR(kernel='poly', C=1e3, degree=2) # Polynomial kernel (degree 2)

    # Fit each model on the data
    svr_rbf.fit(dates, prices)
    svr_lin.fit(dates, prices)
    svr_poly.fit(dates, prices)

    return svr_rbf, svr_lin, svr_poly

# Visualize predictions from each model
def visualize_results(dates, prices, svr_rbf, svr_lin, svr_poly):
    plt.scatter(dates, prices, color='black', label='Data')  # Original data points
    plt.plot(dates, svr_rbf.predict(dates), color='red', label='RBF Model')
    plt.plot(dates, svr_lin.predict(dates), color='green', label='Linear Model')
    plt.plot(dates, svr_poly.predict(dates), color='blue', label='Polynomial Model')
    
    # Add plot labels and title
    plt.xlabel('Year')
    plt.ylabel('Price')
    plt.title('Support Vector Regression Models')
    plt.legend()
    plt.show()

# Predict prices using trained models
def predict_price(svr_rbf, svr_lin, svr_poly, x):
    # Reshape x to match the model's input shape
    x = np.array([[x]])
    # Get predictions from each model
    rbf_pred = svr_rbf.predict(x)[0]
    lin_pred = svr_lin.predict(x)[0]
    poly_pred = svr_poly.predict(x)[0]
    
    return rbf_pred, lin_pred, poly_pred

# Execute the workflow
dates, prices = load_and_preprocess_data(filename)
svr_rbf, svr_lin, svr_poly = train_svr_models(dates, prices)
visualize_results(dates, prices, svr_rbf, svr_lin, svr_poly)
predicted_price = predict_price(svr_rbf, svr_lin, svr_poly, 2024)
print("Predicted prices (RBF, Linear, Polynomial):", predicted_price)
